In [1]:
import pandas as pd
import numpy as np 
import json
import os
from pprint import pprint
from konlpy.tag import Mecab

### data 추출 - 예제 코드입니다. 

- 아래 데이터셋은 0이 19000개 1이 500개정도 이루어진 불균형한 레이블 구성의 데이터셋입니다. 어디까지나 테스트용 리뷰셋입니다.

In [2]:
data1 = pd.read_excel('샴푸샴푸세정력.xlsx')
data2 = data1.loc[:,['review','pos']]
data2.dropna(subset=['review'],inplace=True)

### sklearn - data split

In [3]:
from sklearn.model_selection import train_test_split

class_df = data2['pos']
feature_df = data2.drop(['pos'], axis=1, inplace=False)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
X_train.shape, X_test.shape

((13917, 1), (5965, 1))

## 리뷰데이터 -> 리스트 /// because of vector화

In [4]:
X_train_list = []
for review in X_train['review']:
    X_train_list.append(review)
    
X_test_list = []
for review in X_test['review']:
    X_test_list.append(review)

In [5]:
X_train_list[1]

'넘좋아요\n저렴하게구입했네요'

## word2vec - gensim

In [23]:
r = open('mycorpus_example.txt', mode='r', encoding='utf-8')
reviews = r.readlines()

http://openuiz.blogspot.com/2016/07/mecab-ko-dic.html

In [24]:
from konlpy.tag import Mecab  
mecab=Mecab()
#fread = open('mycorpus_example.txt', mode='r', encoding='utf-8')

n=0
result = []

for review in reviews:
    tokenlist = mecab.pos(review)
    temp=[]
    for word in tokenlist:
        # NNP: 고유명사, NNG:일반명사, MAG:일반 부사, VA:형용사,EC:연결어미, NR:수사
        if word[1] in ["MAG","NNG","VA+EC","VA","NNG","NR"]: 
            temp.append((word[0])) # 해당 단어를 저장함

    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
        result.append(temp) # 결과에 저장

In [25]:
len(result) ,result[:3]

(23056,
 [['기존', '상품', '대비', '가격', '부담', '할인', '높', '만족', '사용', '정가', '구매', '아쉬워'],
  ['뽀득뽀득', '성분', '좋', '샴푸', '거품', '너무', '안', '세정력', '뽀득뽀득', '좋', '좋'],
  ['매번', '거품', '풍', '넝', '맘']])

In [26]:
import keras
from keras.preprocessing.text import Tokenizer

# 가장 빈도가 높은 10,000개의 단어만 선택하도록 Tokenizer 객체를 만듭니다.
tokenizer = Tokenizer(num_words=10000)
# 단어 인덱스를 구축합니다.
tokenizer.fit_on_texts(reviews)

# 문자열을 정수 인덱스의 리스트로 변환합니다.
sequences = tokenizer.texts_to_sequences(reviews)

# 직접 원-핫 이진 벡터 표현을 얻을 수 있습니다.
# 원-핫 인코딩 외에 다른 벡터화 방법들도 제공합니다!
one_hot_results = tokenizer.texts_to_matrix(reviews, mode='count')

# 계산된 단어 인덱스를 구합니다.
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

print(sequences[1])

Using TensorFlow backend.


Found 49395 unique tokens.
[4346, 210, 265, 4, 114, 28, 3, 1036, 311, 954, 1214, 1096, 2713, 13, 8707, 126, 1]


In [27]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=500, window=3, min_count=10,workers=4, sg=0)

In [28]:
# most_similar 함수는 코사인 유사도를 구해줍니다. 
model_result1=model.wv.most_similar("뽀득")
print(model_result1)

[('달라요', 0.9976422786712646), ('전혀', 0.9972014427185059), ('탓', 0.9971702098846436), ('약하', 0.9967210292816162), ('마사지', 0.9963985681533813), ('세정', 0.9958658218383789), ('축', 0.9957591891288757), ('가볍', 0.9956486225128174), ('확', 0.9953948855400085), ('덜하', 0.9951424598693848)]


In [29]:
model_result1=model.wv.most_similar("좋")
print(model_result1)

[('일단', 0.930509090423584), ('너무', 0.9155619144439697), ('향', 0.9055719375610352), ('요향', 0.9022431373596191), ('참', 0.8925585746765137), ('향기', 0.8913396596908569), ('향도', 0.8912991881370544), ('맘', 0.8906192183494568), ('성분', 0.8752028346061707), ('우선', 0.873571515083313)]


### word2vec 모델 저장 및 로드 

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
model.wv.save_word2vec_format('test_01') # 모델 저장
loaded_model=KeyedVectors.load_word2vec_format("test_01") # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("좋")
print(model_result)

## 2. keras - 감성분석

In [22]:
mecab.pos("세일해서 구매했어요")

[('세일', 'NNG'), ('해서', 'XSV+EC'), ('구매', 'NNG'), ('했어요', 'XSV+EP+EC')]

In [10]:
import json
import os
from pprint import pprint
from konlpy.tag import Mecab

mecab = Mecab()

def tokenize(doc):
    # NNP: 고유명사, NNG:일반명사, MAG:일반 부사, VA:형용사,EC:연결어미, NR:수사
    tokenlist = mecab.pos(doc)
    return ['/'.join(word) for word in tokenlist if word[1] in ["MAG","NNG","VA+EC","VA","NNG","NR","VV"]]
            #["MAG","NNG","VA+EC","VA","NNG","NR"]]

if os.path.isfile('train_docs5.json'):
    with open('train_docs5.json') as f:
        train_docs = json.load(f)
    with open('test_docs5.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row),label ) for row,label in zip(X_train['review'],y_train)]
    test_docs = [(tokenize(row),label ) for row,label in zip(X_test['review'],y_test)]
    # JSON 파일로 저장
    with open('train_docs5.json', 'w', encoding="utf-8") as make_file:
        json.dump(str(train_docs), make_file, ensure_ascii=False, indent="\t")
    with open('test_docs5.json', 'w', encoding="utf-8") as make_file:
        json.dump(str(test_docs), make_file, ensure_ascii=False, indent="\t")

# 예쁘게(?) 출력하기 위해서 pprint 라이브러리 사용
pprint(train_docs[0])

(['최고/NNG',
  '바티스트/NNG',
  '성비/NNG',
  '좋/VA',
  '좋/VA',
  '드라이/NNG',
  '샴푸/NNG',
  '없/VA',
  '같/VA',
  '구매/NNG',
  '기억/NNG',
  '안/MAG',
  '머리/NNG',
  '길/VA',
  '매일/MAG',
  '감기/NNG',
  '늘/MAG',
  '유용/NNG',
  '사용/NNG'],
 0)


In [11]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

import nltk
text = nltk.Text(tokens,name='세정력')
print(text)

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

134953
<Text: 세정력>
134953
4670
[('좋/VA', 12868),
 ('샴푸/NNG', 4438),
 ('구매/NNG', 3675),
 ('사용/NNG', 3603),
 ('잘/MAG', 3256),
 ('향/NNG', 3126),
 ('같/VA', 3108),
 ('두피/NNG', 2534),
 ('제품/NNG', 2362),
 ('머리/NNG', 2264)]


In [12]:
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [13]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [14]:
x_val = x_train[:2000]
partial_x_train = x_train[2000:]
y_val = y_train[:2000]
partial_y_train = y_train[2000:]

In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(4670,))) # ERROR 발생시 - inputshape 맞춰주라는 숫자로 바꿔주세요 
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

history = model.fit(x_train, y_train, epochs=10, batch_size=512,validation_data=(x_val,y_val))
results = model.evaluate(x_test, y_test)

Train on 13917 samples, validate on 2000 samples
Epoch 1/10
13917/13917 [==============================] - 1s 84us/sample - loss: 0.2909 - binary_accuracy: 0.9843 - val_loss: 0.1691 - val_binary_accuracy: 0.9820
Epoch 2/10
13917/13917 [==============================] - 1s 85us/sample - loss: 0.1272 - binary_accuracy: 0.9846 - val_loss: 0.1004 - val_binary_accuracy: 0.9820
Epoch 3/10
13917/13917 [==============================] - 1s 80us/sample - loss: 0.0712 - binary_accuracy: 0.9846 - val_loss: 0.0473 - val_binary_accuracy: 0.9820
Epoch 4/10
13917/13917 [==============================] - 1s 66us/sample - loss: 0.0378 - binary_accuracy: 0.9866 - val_loss: 0.0246 - val_binary_accuracy: 0.9915
Epoch 5/10
13917/13917 [==============================] - 1s 63us/sample - loss: 0.0231 - binary_accuracy: 0.9925 - val_loss: 0.0162 - val_binary_accuracy: 0.9965
Epoch 6/10
13917/13917 [==============================] - 1s 70us/sample - loss: 0.0154 - binary_accuracy: 0.9955 - val_loss: 0.0108 - v

In [17]:
results

[0.07912666127738205, 0.98826486]

In [18]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))

In [19]:
predict_pos_neg("뽀득하고 세정이 잘 되는거 같아요.")

[뽀득하고 세정이 잘 되는거 같아요.]는 53.76% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [20]:
predict_pos_neg("세정력 좋아요 향은 별로네요 두피가 시원해요")

[세정력 좋아요 향은 별로네요 두피가 시원해요]는 54.78% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [21]:
predict_pos_neg("향이 좋습니다 세정력은 별로에요 이제 안써요")

[향이 좋습니다 세정력은 별로에요 이제 안써요]는 99.73% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;



In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1,len(loss) +1)

plt.plot(epochs, loss, 'bo', label='Traning loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()
acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()